Clase 7
===

* Ajustar modelo a variables experimentales
* Encontrar la solución más semejante a datos experimentales


Medir calidad de las simulaciones
--------------------------------------

Si conocemos el valor de uno o varios flujos metabólicos podemos ajustar nuestro modelo a estos datos. Hay varias maneras de hacer esto, la primera es simplemente fijar el valor de los flujos como veremos a continuación. 

A modo de ejemplo usaremos *Saccharomyces cerevisae* iMM904 [(descargar aquí)](https://github.com/modcommet/Clases/blob/master/iMM904.xml) y los flujos experimentales compilados por [García et al 2012](https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0043006)


In [1]:
# Let's import the yeast model
import cobra
model = cobra.io.read_sbml_model("iMM904.xml")

In [2]:
# And check the solution using default parameters
solution=model.optimize()
model.summary()

IN FLUXES          OUT FLUXES        OBJECTIVES
-----------------  ----------------  ----------------------
glc__D_e  10       co2_e   18        BIOMASS_SC5_...  0.288
o2_e       2       etoh_e  15.8
nh4_e      1.61    h2o_e    5.64
pi_e       0.0569  h_e      1.45
so4_e      0.0223  for_e    0.00149


In [3]:
# Now, let's modify the ethanol flux
# First, lets find the reactions associated with ethanol
import re

def buscarMetabolito(nombreMetabolite):
    metabolites=[]
    for metabolite in model.metabolites:
        if re.match(nombreMetabolite,metabolite.name,re.IGNORECASE):# and re.match("_e",metabolite.name): 
            metabolites.append(metabolite)
    if len(metabolites)==0:
        print "Not found metabolite"
        return None
    else: #print "2 lets find the reactions associated with the name"
        for metabolite in metabolites:
            print "=================================="
            print metabolite.name, metabolite.id
            print "=================================="
            for reaction in model.metabolites.get_by_id(metabolite.id).reactions:
                print reaction
result=buscarMetabolito("^ethanol$")

Ethanol etoh_c
ETOHt: etoh_e <=> etoh_c
ACHLE3: aces_c + h2o_c --> ac_c + etoh_c + h_c
OHACT1: accoa_c + etoh_c --> aces_c + coa_c
ALCD2x_copy1: etoh_c + nad_c --> acald_c + h_c + nadh_c
ALCD2x_copy2: etoh_c + nad_c <=> acald_c + h_c + nadh_c
ALCD2ir: acald_c + h_c + nadh_c --> etoh_c + nad_c
ETOHtm: etoh_c <=> etoh_m
Ethanol etoh_e
ETOHt: etoh_e <=> etoh_c
EX_etoh_e: etoh_e --> 
Ethanol etoh_m
ALCD2irm: acald_m + h_m + nadh_m --> etoh_m + nad_m
ETOHtm: etoh_c <=> etoh_m


In [4]:
# Fix the ethanol reaction flux and see how it affects the solution
# Fix a range for the ethanol reaction flux around +/-10% of the observed value
value=3.632
model.reactions.get_by_id("EX_etoh_e").upper_bound=value+0.1*value 
model.reactions.get_by_id("EX_etoh_e").lower_bound=value-0.1*value

solution=model.optimize()
model.summary()


IN FLUXES          OUT FLUXES          OBJECTIVES
-----------------  ------------------  ----------------------
glc__D_e  10       co2_e     11.9      BIOMASS_SC5_...  0.247
o2_e       2       h2o_e      9.03
nh4_e      1.38    h_e        7.25
pi_e       0.0488  btd_RR_e   4.81
so4_e      0.0191  etoh_e     4
                   succ_e     3
                   for_e      0.00128


Es posible que errores de medición resulten en datos de flujos metabólicos que no son estequiometricamente compatibles. Para estos casos, una alternativa es fijar un rango de valores en lugar de un valor exacto. Por ejemplo:

Ejercicio
---------

Usando los valores experimentales compilados por [García et al 2012](https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0043006) desde la referencia [20] (ver la columna **References** en Table_S1.doc):

1. Fija la reacción de consumo de glucosa, succinato, ethanol, y piruvato a datos experimentales (haciendo 1 esto debería ser trivial).

Evaluación de la calidad de las predicciones
------------------

Para evaluar que tan cercana es la simulación de FBA a los datos experimentales podemos usar la norma Euclideana la cual mide la distancia entre dos puntos en el hiperespacio:
    
\begin{align}
    \mbox{Norma Euclideana}=\sum_i (x_i-y_i)^2
\end{align}

La cual puede ser escrita usando notación vectorial:

\begin{align}
    \mbox{Norma Euclideana} = (x-y)^T(x-y)
\end{align}


Ejercicio
-----------

Escribe una función para determinar la distancia entre los datos simulados y los experimentales (usa los datos de la Ref 20) usando la norma Euclideana como métrica.


Optimización en dos niveles
-------------------------------

Cuando un FBA tiene multiples soluciones es de interes encontrar aquella que más se asemeje a datos experimentales. Solo en ese caso sería justo evaluar la capacidad de predicción del modelo. Esto se puede hacer formulando una problema de optimización en dos niveles. Mientras que en el nivel más interno se busca maximizar biomasa (esta es la formulación regular de FBA), en el nivel más externo se minimiza la distancia entre flujos simulados y los experimentales.

\begin{align}
\mbox{min}\ & \sum_{i \in I} ||x_i-y_i|| \\
\mbox{s.a.} & \\
&\mbox{max}\  f(x)=c^Tx \\
&\ \mbox{s.a.}  \\
&\ Sx=0 \\
&\ LB<=x<=UB
\end{align}

En donde $\sum_{i \in I} ||x_i-y_i||$ corresponde a una métrica arbitraria, por ejemplo la norma Euclideana (otras opciones puede ser la suma de las diferencias en valores absolutos).

¿Cómo se puede resolver un problema de optimización que depende de otro problema de optimización? Respuesta, usando la [teoría de dualidad](https://en.wikipedia.org/wiki/Linear_programming#Duality) de la programación lineal. En terminos concretos, si tenemos un problema en formato estandar que llamaremos 'primal':

\begin{align}
\mbox{min}\  & c^Tx \\
\mbox{s.a.}&  \\
&\ Ax=b \\
&\ x \ge 0
\end{align}

entonces existe un problema 'dual':

\begin{align}
\mbox{max}\  & b^Ty \\
\mbox{s.a.} & \\
&\ A^Ty \le c
\end{align}

Si existe una solución del primero entonces existe una solución en el segundo y el valor de la función objetivo es el mismo. Esto es útil en un problema de optimización anidado ya que podemos transformar el problema de optimización interno en el siguiente conjunto de restricciones:

\begin{align}
& c^Tx = b^Ty \\
&\ Ax=b \\
&\ A^Ty \le c \\
&\ x \ge 0
\end{align}

Para aplicar esta técnica a FBA primero debemos transformar la formulación de FBA en el formato estandar, lo cual veremos en el pizarron como hacerlo.

Tarea
-------

Estudia el uso de esta técnica en el paper de [Burgard et al 2003](http://onlinelibrary.wiley.com/doi/10.1002/bit.10803/epdf).